In [49]:
from sklearn.externals.joblib import Memory
from sklearn.datasets import load_svmlight_file
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random

eta = 0.01 # Learning Rate
iter = 1000 # Iteration times
accuracy = 0.001 # If loss<accuracy , then stop iteration
mini_batch_percent = 0.1

m_train = 32561 # Amount of training data
m_test = 16281 # Amount of testing data
features=123 # Fearures of dataset

#Initialize arrays
w= [1]*features #All zero initialization

#used to stastic and draw graph
iter_num = [0]*iter;
loss_train  = [0]*iter;
loss_test  = [0]*iter;
acc_train = [0]*iter;
acc_test = [0]*iter;

#Use to get dataset from file
def get_data():
    data_train = load_svmlight_file("dataset\\a9a",n_features=features)
    X_train = data_train[0].toarray()
    y_train = data_train[1]
    data_test = load_svmlight_file("dataset\\a9a.t",n_features=features)
    X_test = data_test[0].toarray()
    y_test = data_test[1]
    return X_train, y_train, X_test, y_test

#The linear model, hw(x) = g(w.T * x)
def hypothesis(x):
    global w
    z =   np.mat(x) * np.mat(w).T
    z = z[0,0]
    print("z:",z);
    result = 1 / (1+(np.e**(-z)))
    return result

#Loss function 
def loss(m,X,y):
    return sum (np.log(1+np.e**(-(X * w.T)*y))) / m

#mini -batch loss function  
def SGD_loss(m,X,y):
    sum = 0.0
    sample_num =math.ceil( m * mini_batch_percent)
    target = np.random.permutation(m)[0:sample_num]
    
    X_new  = X[ target: ]
    y_new = y[target]
    
    return sum (np.log(1+np.e**(-y_new*(X_new * w.T)))) / m

def update(m,X,y):
    global w
    w = w - eta * grad(m,X,y)

#Derivative the Loss function to get Gradient (G) of theta j
def grad(m,X,y):
    sum = np.zeros(features);
    for i in range (0,m):
        sum +=  (hypothesis(X[i]) - y[i])*X[i]
    sum = sum/m
    return sum

#Train and validate 
#def trainAndTest():
    
        
            
def information():
    print("Loss of train:", loss_train)
    print("Loss of valitation",loss_test)

    
#test

a=[1,2,3]
b=[1,1,2]
a = np.mat(a)
b = np.mat(b)
c=  a * b.T
print(c[0,0])


#Main 
X_train, y_train, X_test, y_test = get_data()
print(hypothesis(X_train[0]))
for i in range(0,iter):
    print("loss:",loss(m_train,X_train,y_train))
    update(m_train,X_train,y_train)


#Print the information and draw graphs
print("Features:",w)
fig, ax = plt.subplots()
ax.plot(iter_num, loss_train,color = 'm', label='loss of train')
ax.plot(iter_num, loss_test, color = 'c', label='loss of validation')
plt.legend(bbox_to_anchor=[1, 1])  
ax.set_xlabel('Iteration times')
ax.set_ylabel('Loss')
plt.show()  

fig2, ax2 = plt.subplots()
ax2.plot(iter_num, acc_train,color = 'm', label='Acc of train')
ax2.plot(iter_num, acc_test, color = 'c', label='Acc of validation')
plt.legend(bbox_to_anchor=[1, 1])  
ax2.set_xlabel('Iteration times')
ax2.set_ylabel('Accuracy Rate')
plt.show()


9
z: 14.0
0.999999168472


AttributeError: 'list' object has no attribute 'T'